# 第三步骤：优化构型

本节介绍如何使用模拟退火器（SA）和量子退火器（QA）来寻找分子展开的优化构型。

首先，使用**QMUQUBO**对象加载模型文件。

In [ ]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

In [8]:
qmu_qubo_optimize = QMUQUBO.load(model_path)

In [9]:
model_info = qmu_qubo_optimize.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {2}
INFO:root:param: D, value {8}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


我们可以看到这个模型的参数以及相应的取值：M等于2，D等于8，A等于300，hubo_qubo_val等于200。
实际上，在创建模型时为一个参数提供多个值，就可以在这个文件中包含多个模型。

然后，使用 **model_name** 来获取模型进行实验。

In [10]:
# get the model you want to optimize
M = 2
D = 8
A = 300
hubo_qubo_val = 200
model_name = "{}_{}_{}_{}".format(M, D, A, hubo_qubo_val)
method = "pre-calc"

qubo_model = qmu_qubo_optimize.get_model(method, model_name)



本实验中，我们将用QUBO模型进行实验，其中M等于2。

接下来，设置优化参数。

|参数 |说明 |数值 |
|--- |--- |--- |
|method | QUBO问题的退火方法|'dwave-sa'：使用Ocean工具包中的模拟退火器<br>'dwave-qa'：使用量子退火器|
|shots|读取次数，请参阅 [dwave-sa](https://docs.ocean.dwavesys.com/projects/neal/en/latest/reference/generated/neal.sampler.SimulatedAnnealingSampler.sample.html#neal.sampler. SimulatedAnnealingSampler.sample) 和 [dwave-qa](https://amazon-braket-ocean-plugin-python.readthedocs.io/en/latest/_apidoc/braket.ocean_plugin.braket_sampler.html) 了解详情 |1 到 10,000|
|bucket |存储结果的 S3 存储桶 | - |
|prefix | S3 存储桶中的文件夹名称 | - |
|device |运行量子退火的 arn 名称| 'arn:aws:braket:::device/qpu/d-wave/Advantage_system4' <br> 'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6'|

In [11]:
method = 'dwave-sa'

optimizer_param = {}
optimizer_param['shots'] = 1000

sa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:root:use simulated annealer from dimod


In [12]:
sa_optimize_result = sa_optimizer.fit()

INFO:root:fit() ...
INFO:root:dwave-sa save to local
INFO:root:finish save sa_result.pickle


我们可以选择运行SA解决问题：将SA的shots设置为 1000，并将结果保存为本地文件 **./sa_result.pickle**。

我们也可以选择运行QA解决问题：

In [13]:
method = 'dwave-qa'

optimizer_param = {}
optimizer_param['shots'] = 1000
optimizer_param['bucket'] = s3_bucket # the name of the bucket
optimizer_param['prefix'] = prefix # the name of the folder in the bucket
optimizer_param['device'] = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
optimizer_param["embed_method"] = "default"

qa_optimizer = Annealer(qubo_model, method, **optimizer_param)

INFO:root:use quantum annealer arn:aws:braket:::device/qpu/d-wave/Advantage_system4 


将QA的shots数量设置为1000，并且选择[Advantage_System4.1](https://docs.dwavesys.com/docs/latest/doc_physical_properties.html)作为QPU，结果会自动保存到您的存储桶中。

您可以获取任务ID用于后续处理。

In [14]:
# not create annealing task, only embedding logic
qa_optimizer.embed()
# create annealing task
qa_optimize_result = qa_optimizer.fit()

INFO:root:fit() ...
INFO:root:finish save /tmp/qa_result.pickle
INFO:root:_upload_result_json, bucket=amazon-braket-1a222675c751, key=annealer-experiment/89a59fa9-c66e-482d-85e0-6dfce8c54454/qa_result.pickle
INFO:root:dwave-qa save to s3 - 89a59fa9-c66e-482d-85e0-6dfce8c54454: None


In [15]:
qa_task_id = qa_optimizer.get_task_id()
print(f"task id is {qa_task_id}")

task id is 89a59fa9-c66e-482d-85e0-6dfce8c54454


最后，我们可以比较 SA 和 QA 的执行时间：

In [16]:
print(f"dwave-sa run time {sa_optimize_result['time']}")
print(f"dwave-qa run time {qa_optimize_result['time']}")

dwave-sa run time 175.73437476158142
dwave-qa run time 7.57835578918457


结果显示，SA和QA分别需要174.2秒和7.7秒可以完成优化过程。


![OneTimeQA](../../../images/one-time-qa.png)

有时最优结果仅仅出现过一次。这并不是表示错误，它实际上是反应了问题的特征或取决于公式是如何建立的。

因为有不同的线性项和二次项，它们相差许多数量级。如果将A的变化值设置为一些较小的数字，如10或100，那么最优结果出现的次数会变多。
但是，这些答案通常会打破限制。有关更多信息，请查看
[链接](https://support.dwavesys.com/hc/en-us/community/posts/1500000698522-Number-of-occurrences-?input_string=number%20occurance)。